# Pulling a Series of Screenshots into a PowerPoint Presentation

The objective is to create and save a PowerPoint file populated with images from specified folder, optionally matching certain filename criteria.

### References:
- <a href="https://python-pptx.readthedocs.io/en/latest/index.html">python-pptx</a>
- <a href="https://docs.python.org/3/library/os.html">os — Miscellaneous operating system interfaces</a>

In [1]:
import os
from pptx import Presentation
from pptx.util import Inches, Pt
# from pptx.enum.shapes import MSO_SHAPE
import requests as rq
from bs4 import BeautifulSoup as BS

### Building a presentation from screenshots

In [2]:
prs = Presentation()

# setting the things up
prs.slide_width = Inches(13.333)
prs.slide_height = Inches(7.5)
title_slide_layout = prs.slide_layouts[0]
image_slide_layout = prs.slide_layouts[6]

title = 'ปลดล็อก_ ประวัติศาสตร์ คนไทยคือ _ลูกผสม_ จากสยาม'.replace('_', '')
subtitle = 'ขรรค์ชัย-สุจิตต์ ทอดน่องท่องเที่ยว'

# preparing the title slide
slide = prs.slides.add_slide(title_slide_layout)
title_shape = slide.shapes.title
subtitle_shape = slide.placeholders[1]
prs.save(title + '.pptx')

# populating the placeholders
title_shape.text = title
subtitle_shape.text = subtitle

# prs.save(title + '.pptx')

In [3]:
for shape in slide.shapes:
    if shape.is_placeholder:
        phf = shape.placeholder_format
        print('%d, %s' % (phf.idx, phf.type))
    else:
        print('Not a placeholder')

0, CENTER_TITLE (3)
1, SUBTITLE (4)


Turns out too complicated

### Populating a manually created presentation stub (just blank title) with screenshot images

one per slide.

In [4]:
class Screens2PPTX:

    @staticmethod
    def contains_all(string, lst):
        """Returns True if and only if each meaningful (non-empty, longer than one char) element from the list-like lst is contained in the string"""
        for item in lst:
            item = item.strip('": ')
            if len(item) > 1 and item not in string:
                return False
        return True
    
    def __init__(self, pptx_file, image_dir,
                    keystring=''  # all whitespace-separated fragments of this are to be contained in the image file name as eligibility screen
                                  # if left empty, the value of title from pres properties is used automatically -- TOKENIZE?
                ):
        self.pptx_file = pptx_file.replace('\\', '/')
        self.prs = Presentation(pptx_file)
        self.image_dir = image_dir.replace('\\', '/')
        self.keystring = keystring
        self.title = self.pptx_file.rsplit('/', 1)[-1].rsplit('.', 1)[0]
        self.prs.core_properties.title = self.title

    def populate_title(self):
        title_slide = self.prs.slides[0]
        title_shape = title_slide.shapes.title
        title_shape.text = self.title

    def populate_subtitle(self):
        if not url:
            return
        r = rq.get(url, timeout=30)
        r.encoding = 'utf-8'
        soup = BS(r.text)
        channel = soup.find(attrs={'class': 'ytd-channel-name'}).text
        print(channel)


    def build_title_slide(self):
        self.populate_title()
        self.populate_subtitle()  # not yet implemented
        self.prs.save(self.pptx_file)


    def pull_images(self):
        prs = self.prs
        keystring = self.keystring
        print(f'Pulling from {self.image_dir}\ninto {self.pptx_file}')

        if len(prs.slides) > 1:
            proceed = input('WARNING! More than one slide detected. Eligible images will be added to the end. Any symbol to proceed, mere Enter to cancel.')
            if not proceed:
                print('Operation aborted. Please check destination file contents.')
                return

        blank_slide_layout = prs.slide_layouts[6]

        if not keystring:
            keystring = self.title

        image_files = [file for file in os.scandir(image_dir) if self.contains_all(file.name, keystring.split(' ')) and file.name.endswith('.png')]  # TODO: tokenize rather than split?
        image_files.sort(key=lambda x: x.stat().st_ctime)
        print(f'Total eligible images: {len(image_files)}')

        for image in image_files:
            slide = prs.slides.add_slide(blank_slide_layout)
            slide.shapes.add_picture(image.path, 0, 0, prs.slide_width)
            print('.', end="")
        print('\nDone')

        prs.save(pptx_file)

In [5]:
pptx_file = r'F:\User\Learn\ไทยศึกษา\รุ่นเก๋า เล่าเกร็ด\ep.104 ลางร้าย ในพงศาวดาร (ภาค ๑) - [รุ่นเก๋า...เล่าเกร็ด].pptx'
image_dir = r'C:\Users\User\Videos\Captures'
url = ''

x = Screens2PPTX(pptx_file, image_dir)
x.build_title_slide()
x.pull_images()

Pulling from C:/Users/User/Videos/Captures
into F:/User/Learn/ไทยศึกษา/รุ่นเก๋า เล่าเกร็ด/ep.104 ลางร้าย ในพงศาวดาร (ภาค ๑) - [รุ่นเก๋า...เล่าเกร็ด].pptx
Total eligible images: 44
............................................
Done


In [24]:
x.pull_images()

Pulling from C:/Users/User/Videos/Captures
into F:/User/Explore/Space/ISO/Avi Loeb - The Search for Interstellar Objects of Technological Origin.pptx
Total eligible images: 19
...................
Done


In [22]:
x.keystring

'https://www.youtube.com/watch?v=6eXupH1_v9s'

In [15]:
r = rq.get(url, timeout=30)
r.encoding = 'utf-8'
soup = BS(r.text)
channel = soup.find(attrs={'class': 'ytd-channel-name'}).text
print(channel)

AttributeError: 'NoneType' object has no attribute 'text'

In [20]:
r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
slide = x.prs.slides[0]
for shape in slide.shapes:
    if shape.is_placeholder:
        phf = shape.placeholder_format
        print('%d, %s' % (phf.idx, phf.type))
    else:
        print('Not a placeholder')

0, CENTER_TITLE (3)
1, SUBTITLE (4)


In [23]:
title_shape = slide.shapes.title
dir(title_shape.text_frame.paragraphs[0].font)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_element',
 '_rPr',
 'bold',
 'color',
 'fill',
 'italic',
 'language_id',
 'name',
 'size',
 'underline']

In [26]:
title_shape.text_frame.paragraphs[0].font.size

In [28]:
x.prs.core_properties.title

''